## Продвинутое машинное обучение: 
### Домашнее задание 3
#### Студент: Синяев Максим

Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 

Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.
В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски:

In [21]:
from collections import defaultdict, Counter
import random

import numpy as np
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange, tqdm 

import warnings
warnings.filterwarnings('ignore')

In [ ]:
TRANSLATE_TOKENS = '🍄🍅🍆🍇🍈🍉🍊🍋🍌🍍🍎🍏🍐🍑🍒🍓🍔🍕🍖🍗🍘🍙🍚🍛🍜🍝🍞🍟🍠🍡🍢🍣🍤🍥🍦'
alphabet = ' абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

count_vectorizer = CountVectorizer(token_pattern='\\b[а-я\\ ][а-я\\ ]+\\b', vocabulary=list(alphabet), analyzer='char')
tokenizer = count_vectorizer.build_tokenizer()

In [ ]:
def read_file(filename):
    with open(filename) as f:
        text = f.read()
    return text

def count_freq(corpus, use_filter=True):
    if use_filter:
        freq = dict([(key, val) for key, val in Counter(corpus).items() if key in alphabet])
    else:
        freq = dict([(key, val) for key, val in Counter(corpus).items()])
    sum_ = sum(freq.values())
    freq = dict([(key, (val + 1) / (sum_ + len(alphabet))) for key, val in freq.items()])
    return freq

def decode_from_freq(test_corpus, real_freq):
    test_freq = count_freq(test_corpus, use_filter=False)
    real_freq_ar = np.array(list(real_freq.values()))
    tokens = list(real_freq.keys())
    translate_dict = dict()
    for c in test_freq:
        nearest = np.argmin(np.abs(real_freq_ar - test_freq[c]))
        translate_dict[c] = tokens[nearest]
#     print(translate_dict)
        
    decoded = ''.join([translate_dict[c] for c in test_corpus])
    return decoded, translate_dict

def decode_from_freq_bigrams(test_bigrams, real_freq):
    test_freq = count_freq(test_bigrams, use_filter=False)
    real_freq_ar = np.array(list(real_freq.values()))
    tokens = list(real_freq.keys())
    decoded_seq = ''
    for bi in test_bigrams:
        nearest = np.argmin(np.abs(real_freq_ar - test_freq[bi]))
        decoded_seq += tokens[nearest]
        
#     decoded = ''.join([translate_dict[c] for c in test_corpus])
    return decoded_seq

def translate_accuracy(t1, t2):
    sum_right = 0
    for c1, c2 in zip(t1, t2):
        sum_right += int(c1 == c2)
    mean_ = sum_right / len(t1)
    return mean_

#### 1 Реализуйте базовый частотный метод по Шерлоку Холмсу:


#### подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);


In [ ]:
# Для построения статистики возьмем романы "Анну Каренину" и "Войну и мир"
corpus = read_file('AnnaKarenina.txt') + read_file('WarAndPeace.txt')
corpus = ''.join([c for c in corpus.lower() if c in alphabet])

In [ ]:
uniq_symbols = set(corpus)
print(f'Всего уникальных символов в корпусе: {len(uniq_symbols)}')
# Посчитаем частоты для корпусов оставив только уникальные
train_freq = count_freq(corpus)

#### возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;

In [ ]:
# Пример текста который будет зашифрован
test_text = read_file('./example.txt').lower()
# test_text = 'в течение многих часов шерлок холмс сидел согнувшись над стеклянной пробиркой в которой варилось чтото на редкость вонючее голова его была опущена на грудь и он казался мне похожим на странную товцую птицу с тусклыми серыми перьями и черным хохолком итак уотсон сказал он внезапно вы не собираетесь вкладывать свои сбережения в южноафриканские ценные бумаги я вздрогнул от удивления как ни привык я к необычайным способностям холмса это внезапное вторжение в самые тайные мои мысли было совершенно необъяснимым как черт возьми вы об этом узнали спросил я он повернулся на стуле держа в руке дымящуюся пробирку и его глубоко сидящие глаза радостно заблистали признайтесь уотсон что вы совершенно сбиты с толку сказал он признаюсь мне следовало бы заставить вас написать об этом на листочке бумаги и подписаться почему потому что через пять минут вы скажете что все это необычайно просто'
test_text = ' '.join(tokenizer(test_text))
test_text[:40]

In [ ]:
uniq_test_symb = ''.join(set(test_text))
len(uniq_test_symb)
print(f'Unique tokens: {uniq_test_symb}, len: {len(uniq_test_symb)}')

In [ ]:
translate_dict = dict(zip(TRANSLATE_TOKENS[:len(uniq_test_symb)], uniq_test_symb))
inverse_translate_dict = dict([(val, key) for (key, val) in translate_dict.items()])

encoded_seq = ''.join([inverse_translate_dict[i] for i in test_text])
print(f'Закодированное сообщение теперь выглядит так: {encoded_seq[:40]}')

#### расшифруйте их таким частотным методом.

In [9]:
decoded, trans_dict = decode_from_freq(encoded_seq, train_freq)
print(">Перевод:", decoded)
print()
print(">Количество верно определенных символов: ", translate_accuracy(decoded, test_text))

>Перевод: в ореове хкрдеснуеета еаьокова вешнт яндотеча дшоеа у даееа ьснтаеркояо ахсоеавта еажава   века десвоотксгватевч нешеееса хкрщеетснжеркояо днратевч н оснянеавжеояо угрвнтевч уешду   н   яодоу даее нчоьсев н дортсонв дерчтж васнаетов ветатевжеояо аддасата тчшевее вочдуша двч воеееой сачведкн как уоеодваег так н ьндваег нш фйчевчш еадоунеав еакоееженк ртсевг р ксгвжчун отьсоюееегун еачад еадодоьне ровсеуееегш девжтадваеов н ее нуев шворта жто как хто ен ртсаеео дсндававо еяо уаюнеау досачнтевжеуй уртойжнвортж даее уоя удсаввчтж ахсодваеоу дсншнуач еояой акревесатос в то всеуч как сукн еяо уоявн вертн чаднрн в довете дседвоеееач ахсоеавтнка дововжео сечко доюва в дсуяоу еадсаввеенн врведртвне всошдеееой ьовечен ресдща даее ортавнв авнащнй дорве еекотосояо всеуеен дорвчщеееояо нчоьсетеенй еовгш увужюееегш рдороьов сгьеой воввн еа дснуаеку в   яоду ое одуьвнковав додувчсеуй кеняу о тоу как деватж нркурртвеееге ууюкн дсочсажегун жтоьг сгьа уоява вндетж еарточщнш ууш рквочж ровеежег

Расшивровка как и упоминалось получалась так себе, несмотря на относительно большую точность попадания в символы понять ее весьма сложно

### 2
Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:


#### подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;

In [46]:
bigrams = [''.join(i) for i in ngrams(corpus, 2)]
bigrams_freq = count_freq(bigrams, use_filter=False)

In [47]:
# Пример полученных биграм
list(bigrams_freq.items())[:3]

[('ан', 0.004729391009983087),
 ('нн', 0.0028668525655796253),
 ('на', 0.010638574265792285)]

#### проведите тестирование аналогично п.1, но при помощи биграмм.

In [48]:
def decode_from_freq_bigrams(test_bigrams, real_freq):
    test_freq = count_freq(test_bigrams, use_filter=False)
    real_freq_ar = np.array(list(real_freq.values()))
    tokens = list(real_freq.keys())
    decoded_seq = ''
    for i in range(0, len(test_bigrams), 2):
        bi = test_bigrams[i]
        nearest = np.argmin(np.abs(real_freq_ar - test_freq[bi]))
        decoded_seq += tokens[nearest]
        
#     decoded = ''.join([translate_dict[c] for c in test_corpus])
    return decoded_seq

In [49]:
test_bigrams = [''.join(i) for i in ngrams(encoded_seq, 2)]
decoded = decode_from_freq_bigrams(test_bigrams, bigrams_freq)
print(">Перевод:", decoded)
print()
print(">Количество верно определенных символов: ", translate_accuracy(decoded, test_text))

>Перевод:  бер вн  умин гоу лье  вилнон льля ун ля б же ажгое ниеря е ль у оминие  ж б вляе  вмиу нитоляе н ажажажилн кон  важтонее  жажтоилажльноя  омиерь у е ни ж уерльние ажми уь у у ляниние ния илилеря  вляльь  в оерилажни блян томиляль ильу ерь льние н лягонеу ляь лятоми улянен тоу я илилльн аже ерерил вль ольнеере ляль оль жляажь ля ин ил уер иу гольь ерне иаж пгоилилаже  бльлятоу ми ильля ин еру  у п и иу я минеь ажн у  в бе ил ву ниажернелья гоу  уя  ие негоя е н льилере мие  жаж вилногоажкоилминиляя ажляерн не оерилажажгоми жн  ж бль оил ого ж жу  вя еру  ж бгоя ля бя то илягоу илаже мие ил же н нее  жляажто ольу  нн минито ву  ж б влямие илниил вил жно оляу тоне уя ил вн ил бь е аж бажль уто блято верил баже  жажажне оерернеилниилляажниноми нилноя  бя то иу ь  оеражажя ние н у  икоь ни влян ля жмия ми пноерил билляилль иил ие н ль олятонин ерниго о жлями иер о уляажминиля жние я илерер бу н  вмилянен я миниаж ж о бляляу милья льилляе ил же ил бн  иажя ля жниля пажмин еражльу 

С использованием биграм все стало только хуже, что неудевительно, ведь их очень много, а в тестовом тексте не так много символов чтобы набрать достаточно статистики, хоть сколько-нибудь близкой к реальной, поэтому все частотные оценки получаются у нас сильно смещенными

### Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают.
Основная часть задания — в том, как можно их улучшить:


#### предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;


Руководствуясь интуицией и статьей на медиуме https://towardsdatascience.com/applications-of-mcmc-for-cryptography-and-optimization-1f99222b7132 мы можем воспользоваться методом MCMC сэмплирования и с его помощью подобрать подходящий шифр для нашего зашифрованного сообщения, в общем случае алгоритм будет работать следующи образом:
1. Инициализируем случайный ключ и вычисляем для него вероятность того что текст расшифрован правильно
2. Меняем два случайных ключа в шифре местами
3. Расшифровываем сообщение с помощью полученного на прошлом шаге ключа и считаем веростность того что перевод сделан верно p_prop
4. Если оценка веротности правильной расшифровки  текущего шага получилась выше предыдущего, то используем полученный словарь для следущего шага и возвращаемсяко 2 пункту.
5. Если оценка вероятности существования текста расшифрованного новым ключем меньше прошлого то мы берем новый ключ с вероятностью p_prop/p_previous.
1. Повторяем все начиная с шага 2

Чтобы упростить себе вычисления мы будем использовать Log-Likelihood вместо Likelihood, в алгоритме работы под вероятностью имеется ввиду именно LL.

In [50]:
def decode_text_from_key(text, key):
    return ''.join([key[c] for c in text])

In [80]:
def calc_text_prob(text, key, real_freq: dict, alphabet, n_gram_size):
    decoded_text = decode_text_from_key(text, key)
    prob = 0
    for token_idx in range(len(decoded_text) - n_gram_size):
        bi = decoded_text[token_idx: token_idx + n_gram_size]
        bi_prob = real_freq.setdefault(bi, 1 / (len(alphabet) ** n_gram_size))
        prob += np.log(bi_prob)
    return prob


def find_key_mcmc(encrypted_text, alphabet, corpus_ngram_freq, alphabet_freqs=train_freq, n_gram_size=2, n_iters=1000, verbal=False):
    alphabet_list = list(alphabet)
    encrypt_alphabet_list = list(set(encrypted_text))
    
    # Проверка что количество символов совпадает
#     if len(encrypt_alphabet_list) < len(alphabet):
#         # Иначе дропаем все самые непопулярные токены, исходим из предположения что их не может быть больше
#         tokens = np.array(list(alphabet_freqs.keys()))
#         freqs = list(alphabet_freqs.values())        
#         tokens_idx = np.argsort(freqs)[len(alphabet_list) - len(encrypt_alphabet_list):]
#         alphabet = tokens[tokens_idx].tolist()
#         alphabet_list = list(alphabet)
#         print(f"Dropped tokens: {set(tokens) - set(alphabet)}")
    # Проверка
#     assert len(encrypt_alphabet_list) == len(alphabet)
#     random.shuffle(encrypt_alphabet_list)
    key_prev = dict(zip(encrypt_alphabet_list, alphabet))
    prob_prev = calc_text_prob(encrypted_text, key_prev, corpus_ngram_freq, alphabet, n_gram_size)
    
    desc = 'Searching best key with mcmc, NLL = {}'
    t = trange(n_iters, desc=desc.format(prob_prev), disable=(not verbal))
    n_tries_no_change = 0
    for i in t:
        # Chose random idxs
        idx1, idx2 = random.sample(range(len(alphabet)), 2)
        alphabet_list_copy = alphabet_list.copy()
        alphabet_list_copy[idx1], alphabet_list_copy[idx2] = alphabet_list_copy[idx2], alphabet_list_copy[idx1]
        
        key_new = dict(zip(encrypt_alphabet_list, alphabet_list_copy[:len(encrypt_alphabet_list)]))
        prob_new = calc_text_prob(encrypted_text, key_new, corpus_ngram_freq, alphabet, n_gram_size)
        
        # Берем экспоненту чтобы привести вероятность к нормальному виду [0,1], отнимаем потому что логарифм
        p_change = np.exp(prob_new - prob_prev)
        
        # Попытка стабилизировать алгоритм, вроде работает, переходы происходят всегда когда вероятность больше,
        # если застревает уменьшаем порог для перехода, не всегда достигаются значения, но работает стабильнее (нет)
#         if prob_new > (prob_prev / np.exp(n_tries_no_change)) or p_change > random.random():
        if p_change > random.random():

            n_tries_no_change = 0
            alphabet_list = alphabet_list_copy
            key_prev = key_new
            prob_prev = prob_new
        else:
            n_tries_no_change += 1

        if (i + 1) % 10 == 0:
            t.set_description(desc.format(prob_prev))
#     print(f'Best log_prob: {prob_prev}')
    return key_new, prob_prev
                
                
    
    

In [81]:
mcmc_key, _ = find_key_mcmc(encoded_seq, alphabet, bigrams_freq, n_iters=10000, verbal=True)

Searching best key with mcmc, NLL = -11445.700629592133: 100%|██████████| 10000/10000 [00:28<00:00, 350.66it/s]


In [82]:
decoded = decode_text_from_key(encoded_seq, mcmc_key)
print(decoded)
print(">Количество верно определенных символов: ", translate_accuracy(decoded, test_text) * 100, '%')

ке ви кнебявгнотчнислеилм я клеэнютсеутг сньледю илередлиилемотсливя у елбо илкслеилылэлееекнялегнок  сяозклснэжетиюнинолебявфнисотынвя у егтвлснэжете отутилэпи у ечзвэтснэжечнюдрееетеееу д чедлииеть монэетег всо тэеднвжспеклотлис кеэнслснэпи у елгглолслесжюнэннек ьдршледэжек нии цеолькндятеяляеч и гэлизесляетемтгэлизетшейщьнэжюеилг чтилэеиля иныитяевсонэзевеяозэпжчте смо хниизчтеильлдеилг д мтнев кончниизшеднэпслгэли кетеинетчнэешк вслеыс еяляебс еитевсолии еготдлклэ ену ечлхтилчег ольтснэпирщервс цытк вспедлииеч уерголкэжспелбо гэли чеготютчлжеи у целявнэнолс оекес екончжеяляеорятену еч уэтекнвстеьлгтвтекег энснегондк ниилжелбо илкстялед к эпи еонья ег хэлекедору чеилголкэниттеквэндвсктнеко юднии цем эньитевнодфледлиие вслктэелктлфтщег вэнеиня с о у екончнитег вкжании у еть монснитщеи кзшерэрыхниизшевг в м кеозми цеэ кэтеилеготчлиярекеееу дре ие грмэтя клэег грэжоирщеяитуре ес чеяляеднэлспетвярввскниизнечрхятего ьолыизчтеыс мзеозмлеч уэлектднспеилвс жатшечршевяк ьпев эиныизцевкнсе ие

Алгоритм время от времени застревает в локальном минимуме и никак не может оттуда выбраться, давайте напишем цикл в котором будем запускить алгоритм несколько раз и выбирать ключ с наименьшим по модулю NLL

In [83]:
# N_ATTEMPTS = 20

# best_key = None
# best_prob = float('-inf')
# probs_history = list()
# decryption_error_history = list()
# for i in range(N_ATTEMPTS):
#     key, prob = find_key_mcmc(encoded_seq, alphabet, bigrams_freq, n_iters=5000)
#     probs_history.append(prob)
#     decryption_error_history.append(translate_accuracy(decode_text_from_key(encoded_seq, key), test_text))
#     if prob > best_prob:
#         best_key = key
#         best_prob = prob
    

#### 6 Расшифруйте сообщение:


In [84]:
secret_message = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'


In [85]:
best_key = None
best_prob = float('-inf')
for i in tqdm(range(50)):
    mcmc_key_secret, prob = find_key_mcmc(secret_message, alphabet, bigrams_freq, n_iters=10_000)
    if prob > best_prob:
        best_key = mcmc_key_secret
        best_prob = prob

100%|██████████| 50/50 [02:47<00:00,  3.36s/it]


In [86]:
best_prob

-1236.3806359456694

In [87]:
decoded = decode_text_from_key(secret_message, best_key)
print(decoded)

чсли вы вимитч нордальный или поети нордальный тчкст у этого сообщчния который лчгко проеитать скорчч всчго вы всч смчлали правильно и полуеитч даксидальный балл за послчмнчч ечтвчрточ заманич курса хотя кончено я ниечго нч обчщаф


Возможно проблема в том что отбрасываются буквы необходимые для расшиврофки, можно попробовать составить алфавит для расшифровки самим

#### 5 Бонус:
а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

In [88]:
# Посчитаем триграммы и их частоты

trigrams = [''.join(i) for i in ngrams(corpus, 3)]
trigrams_freq = count_freq(trigrams, use_filter=False)

In [89]:
# Пример полученных биграм
list(trigrams_freq.items())[:3]

[('анн', 0.0009612968350036416),
 ('нна', 0.0004097330772146669),
 ('на ', 0.00565832009438342)]

In [90]:
best_key = None
best_prob = float('-inf')
for i in tqdm(range(50)):
    mcmc_key_secret, prob = find_key_mcmc(secret_message, alphabet, trigrams_freq, n_iters=10_000, n_gram_size=3)
    if prob > best_prob:
        best_key = mcmc_key_secret
        best_prob = prob


100%|██████████| 50/50 [02:51<00:00,  3.43s/it]


In [91]:
best_prob

-1725.1100359228005

In [92]:
decoded = decode_text_from_key(secret_message, best_key)
print(decoded)

если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса ъотя конечно я ничего не обещаж
